In [ ]:
#import necessary libraries 
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from io import StringIO
from selenium import webdriver



In [ ]:
# years stats are tracked
years = list(range(1991, 2024))
main_url = "https://www.basketball-reference.com/awards/awards_{}.html"

# retrieve mvp ladder data tables for each year 
for year in years:
    url = main_url.format(year)
    data = requests.get(url)
    time.sleep(10)
    # save data in mvp_data folder
    with open("mvp_data/{}.html".format(year), "w+") as f:
        f.write(data.text)


In [ ]:
with open("mvp_data/1991.html") as f:
    page = f.read()
    
soup = BeautifulSoup(page, 'html.parser')
# remove header of table
soup.find('tr', class_= "over_header").decompose()
# find data table using its id
mvp_table = soup.find(id = "mvp")

In [ ]:
# single table from 1991
mvp_1991 = pd.read_html(StringIO(str(mvp_table)))[0]
mvp_1991.head(1)


In [ ]:
mvp_tables = []

# loop through all year files and store the tables
for year in years:
    with open("mvp_data/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_= "over_header").decompose()
    mvp_table = soup.find_all(id="mvp")[0]
    mvp_df = pd.read_html(StringIO(str(mvp_table)))[0]
    mvp_df["Year"] = year
    tables.append(mvp_df)


In [ ]:
# concatenate the tables
all_mvps = pd.concat(tables)
all_mvps.to_csv("mvps.csv")

In [57]:
# Player Stats URL
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"
# Chrome Driver
driver = webdriver.Chrome()

In [59]:
# Access each stats page
years = list(range(1991, 2024))
for year in years:
    url = player_stats_url.format(year)
    driver.get(url)
    #scroll to bottom of page to get whole table
    driver.execute_script("window.scrollTo(1,10000)")

    with open("player_data/{}.html".format(year), "w+") as f:
        f.write(driver.page_source)

In [73]:
# loop for player stats files each year and add to table
player_stats_df = []

years = list(range(1991, 2024))
for year in years:
    with open("player_data/{}.html".format(year)) as f:
        page = f.read()
    soup = BeautifulSoup(page, 'html.parser')
    # remove table header elements
    soup.find('tr', class_="thead").decompose()
    stats_table = soup.find_all(id="per_game_stats")[0]
    stats_df = pd.read_html(StringIO(str(stats_table)))[0]
    stats_df["Year"] = year
    player_stats_df.append(stats_df)

# concatenate all years' tables
player_stats = pd.concat(player_stats_df)
player_stats.to_csv("player_stats.csv")
    



In [90]:
# Getting Team W/L Record
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

# put html pages into folder
years = list(range(2022, 2024))
for year in years:
    url = team_stats_url.format(year)
    team_data = requests.get(url)
    time.sleep(5)
    with open("team_data/{}.html".format(year), "w+") as f:
        f.write(team_data.text)

In [107]:
# Parse W/L record based on conferences
team_df = []
years = list(range(1991, 2024))
for year in years:
    with open("team_data/{}.html".format(year)) as f:
        page = f.read()

    # Eastern Conf.
    soup = BeautifulSoup(page,'html.parser')
    soup.find('tr', class_= 'thead').decompose()
    east_table = soup.find_all(id="divs_standings_E")[0]
    east_df = pd.read_html(StringIO(str(east_table)))[0]
    east_df["Year"] = year
    east_df["Team"] = east_df["Eastern Conference"]
    del east_df["Eastern Conference"]
    team_df.append(east_df)

    # Western Conf.
    west_table = soup.find_all(id="divs_standings_W")[0]
    west_df = pd.read_html(StringIO(str(west_table)))[0]
    west_df["Year"] = year
    west_df["Team"] = west_df["Western Conference"]
    del west_df["Western Conference"]
    team_df.append(west_df)
    
teams = pd.concat(team_df)



In [113]:
# convert to csv file
teams.to_csv("teams.csv")